In [46]:
import numpy as np
def SOR(S, K, r, q, T, sigma, option_type, Smin, Smax, Ns, Nt, theta, alpha, epsilon):
    
    S = S
    K = K
    r = r
    q = q
    T = T
    sigma = sigma
    option_type = option_type
    is_call = (option_type[0].lower()=='c')
    omega = 1 if is_call else -1
    Ns = int(Ns)
    max_iter = 10*Nt
    Nt = int(Nt)
    dS = (Smax-Smin)/Ns * 1.0
    dt = T/Nt*1.0
    Svec = np.linspace(Smin, Smax, Ns+1)
    Tvec = np.linspace(0, T, Nt+1)
    grid = np.zeros(shape=(Ns+1, Nt+1))
    
    # boundary_condition_(self):
    tau = Tvec[-1] - Tvec;     
    DFq = np.exp(-q * tau)
    DFr = np.exp(-r * tau)
    
    # american option boundary condition should differ from Euro style
    grid[0,  :] = np.maximum(omega*(Svec[0]  - K), 0)
    grid[-1, :] = np.maximum(omega*(Svec[-1] - K), 0)     
    grid[:, -1] = np.maximum(omega*(Svec - K), 0)
    
  # set_coefficient__(self):
    drift = (r-q)*Svec[1:-1]/dS
    diffusion_square = (sigma*Svec[1:-1]/dS)**2

    l = 0.5*(diffusion_square - drift)
    c = -diffusion_square - r
    u = 0.5*(diffusion_square + drift)
    
    
    # matrix_(self):
    A = sp.diags([l[1:], c, u[:-1]], [-1, 0, 1],  format='csc')
    I = sp.eye(Ns-1)
    M1 = I + (1-theta)*dt*A
    
              
    w = alpha
    thedt = theta * dt
    payoff = grid[1:-1, -1]
    m = len(payoff)
    pastval = payoff.copy()
    
    def trigger( oldval, newval, tol, counter, maxIteration ):
        noBreak = 1
        if np.max( np.abs(newval-oldval)/np.maximum(1,np.abs(newval)) ) <= tol:
            noBreak = 0
        elif counter > maxIteration:
            print('The result may not converge.')
            noBreak = 0
        return noBreak
        
    for j in reversed(np.arange(Nt)):
        counter = 0
        noBreak = 1
        newval = pastval.copy()

        z = M1.dot(pastval)

        z[0] += theta*l[0]*dt*grid[0, j] \
             + (1-theta)*l[0]*dt*grid[0, j+1] 
        z[-1] += theta*u[-1]*dt*grid[-1, j] \
              + (1-theta)*u[-1]*dt*grid[-1, j+1] 

        while noBreak:
            counter += 1
            oldval = newval.copy()
            newval[0] = np.maximum( payoff[0], oldval[0] + w/(1-thedt*c[0]) \
                                   *( z[0] - (1-thedt*c[0])*oldval[0] \
                                     + thedt*u[0]*oldval[1]) )
            for k in np.arange(1,m-1):
                newval[k] = np.maximum( payoff[k], oldval[k] + w/(1-thedt*c[k]) \
                                       *( z[k] + thedt*l[k]*newval[k-1] \
                                         - (1-thedt*c[k])*oldval[k] \
                                         + thedt*u[k]*oldval[k+1]) )

            newval[m-1] = np.maximum( payoff[m-1], oldval[m-1] + w/(1-thedt*c[m-1]) \
                                     *( z[m-1] + thedt*l[m-1]*newval[m-2] \
                                       - (1-thedt*c[m-1])*oldval[m-1]) )

            noBreak = trigger( oldval, newval, epsilon, counter, max_iter )

        pastval = newval.copy()
        grid[1:-1, j] = pastval

    return spi.splev( S, tck = spi.splrep( Svec, grid[:,0], k=3 ) )


In [47]:
# initial
(S, K, r, q, T, sigma, option_type) = (12, 5, 0.1, 0.01, 1, 0.4, 'call')
(Smin, Smax, Ns, Nt) = (0, 4*np.maximum(S,K), 200, 200)
(theta, alpha, epsilon) = (0.5, 1.5, 1e-6)

amer_opt = SOR(S, K, r, q, T, sigma, option_type, Smin, Smax, Ns, Nt, theta, alpha, epsilon)
print(amer_opt)

7.363998157694755
